In [1]:
import spacy
import pandas as pd
import numpy as np
import pickle
import random
import nltk
from pprint import pprint
import operator

In [2]:
from spacy.attrs import LOWER, POS, ENT_TYPE, IS_ALPHA


nlp = spacy.load("en_core_web_sm")

## Finding NN phrases with atleast one JJ

In [3]:
data_files = ["papers_2017_balanced.pickle", "papers_2018_balanced.pickle", "papers_2019_balanced.pickle", "papers_2020_balanced.pickle"]

In [4]:
yearwise_data = []

for ind_file in data_files:
    with open("ICLR data/"+ind_file, "rb") as f:
        yearwise_data.append(pickle.load(f))

In [5]:
total = 0
for data in yearwise_data:
    total += len(data)
print(total)

3446


In [6]:
adjective_phrases_acc = {}
adjective_phrases_reject = {}

bad_count = good_count = 0

for data in yearwise_data:
    for k,v in data.items():
        if "decision" in data[k]["DECISIONS"]:
            if "decision" in data[k]["DECISIONS"]["decision"]["content"]:
                dec = data[k]["DECISIONS"]["decision"]["content"]["decision"].lower()
            else:
                dec = data[k]["DECISIONS"]["decision"]["content"]["recommendation"].lower()
        else:
            dec = data[k]["DECISIONS"]["content"]["decision"].lower()
        if "OFF_REVIEW" in data[k]:
            for review in data[k]["OFF_REVIEW"]:
                try:
                    review_text = review["content"]["review"]
                    doc = nlp(review_text)
                    for nchunk in doc.noun_chunks:
                        nchunk_tags = [t.tag_ for t in nchunk]
                        if "JJ" in nchunk_tags or "JJR" in nchunk_tags or "JJS" in nchunk_tags:
                            cleaned_chunk = " ".join([tok.lemma_.lower() for tok in nchunk])
                            
                            if "accept" in dec:
                                if cleaned_chunk in adjective_phrases_acc:
                                    adjective_phrases_acc[cleaned_chunk] += 1
                                else:
                                    adjective_phrases_acc[cleaned_chunk] = 1
                            else:
                                if cleaned_chunk in adjective_phrases_reject:
                                    adjective_phrases_reject[cleaned_chunk] += 1
                                else:
                                    adjective_phrases_reject[cleaned_chunk] = 1
                    good_count += 1
                except Exception as ex:
                    bad_count += 1
        else:
            print("OFF_REVIEW not found for: ", k)
print(good_count)
print(bad_count)

OFF_REVIEW not found for:  ry5wc1bCW
10454
0


In [7]:
# adjective_phrases_acc
sorted_adj_phrases_acc = sorted(adjective_phrases_acc.items(), key=lambda item: item[1], reverse=True)

In [10]:
sorted_adj_phrases_acc[0:50]

[('neural network', 505),
 ('the experimental result', 408),
 ('previous work', 327),
 ('minor comment', 283),
 ('more detail', 239),
 ('the main contribution', 227),
 ('experimental result', 210),
 ('adversarial example', 208),
 ('related work', 206),
 ('prior work', 205),
 ('deep neural network', 187),
 ('the main text', 186),
 ('deep learning', 179),
 ('the other hand', 166),
 ('future work', 165),
 ('other method', 163),
 ('the experimental section', 158),
 ('deep network', 158),
 ('a neural network', 152),
 ('the main idea', 142),
 ('-pron- main concern', 139),
 ('adversarial training', 137),
 ('the related work', 134),
 ('the empirical result', 130),
 ('adversarial attack', 125),
 ('detailed comment', 116),
 ('an important problem', 115),
 ('a good job', 114),
 ('the main paper', 112),
 ('other word', 108),
 ('the neural network', 106),
 ('well performance', 105),
 ('the related work section', 102),
 ('the theoretical result', 97),
 ('good result', 92),
 ('the experimental evalua

In [13]:
# adjective_phrases_reject
sorted_adj_phrases_rej = sorted(adjective_phrases_reject.items(), key=lambda item: item[1], reverse=True)

In [14]:
sorted_adj_phrases_rej[0:40]

[('neural network', 533),
 ('the experimental result', 432),
 ('previous work', 349),
 ('the main contribution', 257),
 ('minor comment', 252),
 ('experimental result', 237),
 ('more detail', 226),
 ('adversarial example', 219),
 ('prior work', 205),
 ('related work', 202),
 ('deep learning', 197),
 ('other method', 186),
 ('the other hand', 186),
 ('-pron- main concern', 179),
 ('the main idea', 169),
 ('a neural network', 163),
 ('deep neural network', 162),
 ('the experimental section', 161),
 ('detailed comment', 155),
 ('the empirical result', 143),
 ('deep network', 140),
 ('adversarial training', 136),
 ('the related work', 121),
 ('the main text', 120),
 ('other word', 120),
 ('well performance', 113),
 ('adversarial attack', 109),
 ('the neural network', 107),
 ('-pron- current form', 106),
 ('generative model', 106),
 ('the experimental evaluation', 100),
 ('the objective function', 97),
 ('the related work section', 93),
 ('an important problem', 92),
 ('the same time', 90),

In [9]:
c = 0
t = 0

for data in yearwise_data:
    for k,v in data.items():
        t += 1
        try:
            dec = data[k]["DECISIONS"]["decision"]["content"]["decision"]
        except Exception as ex:
            try:
                if "decision" in data[k]["DECISIONS"]:
                    dec = data[k]["DECISIONS"]["decision"]["content"]["recommendation"]
                elif "content" in data[k]["DECISIONS"]:
                    dec = data[k]["DECISIONS"]["content"]["decision"]
            except Exception as e:
                c+=1
                print(data[k]["DECISIONS"].keys())
                pprint(data[k]["DECISIONS"])
                print("done")
                break
print(c)
print(t)

0
3446


In [35]:
# list(yearwise_data[2].keys())[0:5]
yearwise_data[2]["H1g4k309F7"]["DECISIONS"]["decision"]["content"]

{'confidence': '4: The area chair is confident but not absolutely certain',
 'metareview': 'The paper proposes a novel way to ensemble multi-class or multi-label models\nbased on a Wasserstein barycenter approach. The approach is theoretically\njustified and obtains good results. Reviewers were concerned with time\ncomplexity, and authors provided a clear breakdown of the complexity.\nOverall, all reviewers were positives in their scores, and I recommend accepting the paper.',
 'recommendation': 'Accept (Poster)',
 'title': 'meta-review'}

In [28]:
data["ryxgsCVYPr"]["DECISIONS"]

{'decision': {'cdate': 1576798719989,
  'content': {'comment': 'This paper extracts a list of conditions from the question, each of which should be satisfied by the candidate answer generated by an MRC model. All reviewers agree that this approach is interesting (verification and validation) and experiments are solid. One of the reviewers raised concerns are promptly answered by authors, raising the average score to accept. \n',
   'decision': 'Accept (Poster)',
   'title': 'Paper Decision'},
  'ddate': None,
  'details': {'replyCount': 0},
  'forum': 'ryxgsCVYPr',
  'id': 'bmukdcHZsa',
  'invitation': 'ICLR.cc/2020/Conference/Paper1307/-/Decision',
  'nonreaders': [],
  'number': 1,
  'original': None,
  'readers': ['everyone'],
  'referent': None,
  'replyto': 'ryxgsCVYPr',
  'signatures': ['ICLR.cc/2020/Conference/Program_Chairs'],
  'tcdate': 1576798719989,
  'tmdate': 1576800916570,
  'writers': ['ICLR.cc/2020/Conference/Program_Chairs']}}

In [19]:
len(adjective_phrases_yearwise)

138328

In [24]:
sorted_adj_phrases = sorted(adjective_phrases_yearwise.items(), key=lambda item: item[1], reverse=True)

In [26]:
sorted_adj_phrases[0:10]

[('neural network', 1038),
 ('the experimental result', 840),
 ('previous work', 676),
 ('minor comment', 535),
 ('the main contribution', 484),
 ('more detail', 465),
 ('experimental result', 447),
 ('adversarial example', 427),
 ('prior work', 410),
 ('related work', 408)]

In [13]:
yearwise_data[1]["ry5wc1bCW"]

NameError: name 'yearwise_data' is not defined

## Scratch


In [34]:
dir(spacy.attrs)
dir(POS)

['__abs__', '__add__', '__and__', '__bool__', '__ceil__', '__class__', '__delattr__', '__dir__', '__divmod__', '__doc__', '__eq__', '__float__', '__floor__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getnewargs__', '__gt__', '__hash__', '__index__', '__init__', '__int__', '__invert__', '__le__', '__lshift__', '__lt__', '__mod__', '__mul__', '__ne__', '__neg__', '__new__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdivmod__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rlshift__', '__rmod__', '__rmul__', '__ror__', '__round__', '__rpow__', '__rrshift__', '__rshift__', '__rsub__', '__rtruediv__', '__rxor__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__truediv__', '__trunc__', '__xor__', 'bit_length', 'conjugate', 'denominator', 'from_bytes', 'imag', 'numerator', 'real', 'to_bytes']

In [8]:
nlp = spacy.load("en_core_web_sm")

In [4]:
doc = nlp("The quick brown fox jumps over the lazy dog.")

In [5]:
for tok in doc:
    print(tok.text, tok.pos_, tok.dep_)

The DET det
quick ADJ amod
brown ADJ amod
fox NOUN compound
jumps NOUN ROOT
over ADP prep
the DET det
lazy ADJ amod
dog NOUN pobj
. PUNCT punct


In [6]:
for tok in doc.noun_chunks:
    print(tok)

The quick brown fox jumps
the lazy dog


In [4]:
review_data = None
with open("ICLR data/papers_2020_balanced.pickle", "rb") as f:
    review_data = pickle.load(f)

In [5]:
list(review_data.keys())[0:10]

['ryxgsCVYPr',
 'rJe8pxSFwr',
 'SJldu6EtDS',
 'r1xGP6VYwH',
 'H1x9004YPr',
 'H1eH9hNtwr',
 'BJxsrgStvr',
 'SJlYqRNKDS',
 'BklIxyHKDr',
 'B1xm3RVtwB']

In [6]:
pprint(review_data["H1gNOeHKPS"]["OFF_REVIEW"][0]["content"]["review"])

('The authors propose the Neural Multiplication Unit (NMU), which can learn to '
 'solve a family of arithmetic operations using -, + and * atomic operations '
 'over real numbers from examples. They show that a combination of careful '
 'initialization, regularization and structural choices allows their model to '
 'learn more reliably and efficiently than the previously published Neural '
 'Arithmetic Logic Unit.\n'
 '\n'
 'The NALU consists of two additive sub-units in the real and log-space '
 'respectively, which allows it to handle both additions/subtractions and '
 'multiplications/divisions, and combines them with a gating mechanism. The '
 'NMU on the other hand simply learns a product of affine transformations of '
 'the input. This choice prevents the model from learning divisions, which the '
 'authors argue made learning unstable for the NALU case, but allows for an a '
 'priori better initialization and dispenses with the gating which is '
 'empirically hard to learn. The

In [9]:
doc = nlp(review_data["H1gNOeHKPS"]["OFF_REVIEW"][0]["content"]["review"])

In [12]:
for tok in doc.noun_chunks:
    print(tok.lemma_.lower())

the author
the neural multiplication unit
nmu
a family
arithmetic operation
and * atomic operation
real number
example
-pron-
a combination
careful initialization
regularization
structural choice
-pron- model
the previously publish neural arithmetic logic unit
the nalu
two additive sub - unit
the real and log - space
-pron-
both addition / subtraction
multiplication
/ division
-pron-
a gate mechanism
the nmu
the other hand
a product
affine transformation
the input
this choice
the model
learning division
the author
the nalu case
an a priori well initialization
dispense
the gating
the departure
the nalu architecture
significant improvement
the consider application
extrapolation
the training domain
the paper
one notable exception
the form
the loss function
the paper
the scope
the work
the author
the application
-pron-
a wide context
division
a sufficient justification
-pron-
the author
future work
the propose unit
at least 10 k example
a single expression
few than 10 input
the success rat

In [10]:
for tok in doc.noun_chunks:
    print(tok, [t.tag_ for t in tok])

The authors ['DT', 'NNS']
the Neural Multiplication Unit ['DT', 'NNP', 'NNP', 'NNP']
NMU ['NNP']
a family ['DT', 'NN']
arithmetic operations ['JJ', 'NNS']
and * atomic operations ['CC', 'NFP', 'JJ', 'NNS']
real numbers ['JJ', 'NNS']
examples ['NNS']
They ['PRP']
a combination ['DT', 'NN']
careful initialization ['JJ', 'NN']
regularization ['NN']
structural choices ['JJ', 'NNS']
their model ['PRP$', 'NN']
the previously published Neural Arithmetic Logic Unit ['DT', 'RB', 'VBN', 'NNP', 'NNP', 'NNP', 'NNP']
The NALU ['DT', 'NNP']
two additive sub-units ['CD', 'JJ', 'NN', 'NNS', 'NNS']
the real and log-space ['DT', 'JJ', 'CC', 'NN', 'HYPH', 'NN']
it ['PRP']
both additions/subtractions ['DT', 'NNS', 'SYM', 'NNS']
multiplications ['NNS']
/divisions ['SYM', 'NNS']
them ['PRP']
a gating mechanism ['DT', 'VBG', 'NN']
The NMU ['DT', 'NNP']
the other hand ['DT', 'JJ', 'NN']
a product ['DT', 'NN']
affine transformations ['JJ', 'NNS']
the input ['DT', 'NN']
This choice ['DT', 'NN']
the model ['DT',

In [58]:
for i in range(len(doc)):
    if doc[i].pos_ == "ADJ" and doc[i+1].tag_.startswith("NN"):
        print(doc[i-1], doc[i-1].tag_, doc[i], doc[i+1])

of IN arithmetic operations
* NFP atomic operations
over IN real numbers
of IN careful initialization
and CC structural choices
two CD additive sub
the DT other hand
of IN affine transformations
priori FW better initialization
to IN significant improvements
one CD notable exception
a DT wider context
a DT sufficient justification
a DT plausible way
for IN future work
a DT single expression
or CC efficient architecture


The
authors
propose
the
Neural
Multiplication
Unit
(
NMU
)
,
which
can
learn
to
solve
a
family
of
arithmetic
operations
using
-
,
+
and
*
atomic
operations
over
real
numbers
from
examples
.
They
show
that
a
combination
of
careful
initialization
,
regularization
and
structural
choices
allows
their
model
to
learn
more
reliably
and
efficiently
than
the
previously
published
Neural
Arithmetic
Logic
Unit
.



The
NALU
consists
of
two
additive
sub
-
units
in
the
real
and
log
-
space
respectively
,
which
allows
it
to
handle
both
additions
/
subtractions
and
multiplications
/
divisions
,
and
combines
them
with
a
gating
mechanism
.
The
NMU
on
the
other
hand
simply
learns
a
product
of
affine
transformations
of
the
input
.
This
choice
prevents
the
model
from
learning
divisions
,
which
the
authors
argue
made
learning
unstable
for
the
NALU
case
,
but
allows
for
an
a
priori
better
initialization
and
dispenses
with
the
gating
which
is
empirically
hard
to
learn
.
The
departures
from
the
NALU
architectu

In [22]:
dir(doc)

['_', '__bytes__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__pyx_vtable__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__unicode__', '_bulk_merge', '_py_tokens', '_realloc', '_vector', '_vector_norm', 'cats', 'char_span', 'count_by', 'doc', 'ents', 'extend_tensor', 'from_array', 'from_bytes', 'from_disk', 'get_extension', 'get_lca_matrix', 'has_extension', 'has_vector', 'is_nered', 'is_parsed', 'is_sentenced', 'is_tagged', 'lang', 'lang_', 'mem', 'merge', 'noun_chunks', 'noun_chunks_iterator', 'print_tree', 'remove_extension', 'retokenize', 'sentiment', 'sents', 'set_extension', 'similarity', 'tensor', 'text', 'text_with_ws', 'to_array', 'to_bytes', 'to_disk', 'to_json', 'to_utf8_array', 'user_data', 'user_hooks', 'user_span_hooks', 'u

In [31]:
nparr = doc.to_array([LOWER, POS, ENT_TYPE, IS_ALPHA])

In [38]:
for t in nparr:
    if t[1] == "ADJ"
#     print((t.view()))
#     break

[7425985699627899538                  90                   0
                   1]


TypeError: 'int' object is not callable